In [1]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
%aimport bs_lib.bs_eda

In [2]:
from os.path import join
import pandas as pd
import numpy as np
import bs_lib.bs_eda as beda

## Création du dataset

Chargement et fusion des fichiers csv en un unique dataset

In [15]:
file_to_exclude = ["unclean focus.csv","unclean cclass.csv","focus.csv","cclass.csv",
                   "train_set.csv","val_set.csv","test_set.csv",
                   "train_set_pre_training_processed.csv",
                   "train_set_preprocessed.csv",
                   "train_set_light_preprocessed.csv",
                   "all_set.csv"]
directory = 'dataset/'
all_df = beda.load_all_csv(dataset_path=directory, exclude=file_to_exclude)

columns = all_df['audi'].columns.to_list()

all_set_df = pd.DataFrame(columns=columns)
all_df_train = pd.DataFrame(columns=columns)
all_df_val = pd.DataFrame(columns=columns)
all_df_test = pd.DataFrame(columns=columns)

target = 'price'

loading:['dataset/skoda.csv', 'dataset/ford.csv', 'dataset/vw.csv', 'dataset/audi.csv', 'dataset/toyota.csv', 'dataset/bmw.csv', 'dataset/hyundi.csv', 'dataset/merc.csv', 'dataset/vauxhall.csv']
Parsing dataset/skoda.csv
Parsing dataset/ford.csv
Parsing dataset/vw.csv
Parsing dataset/audi.csv
Parsing dataset/toyota.csv
Parsing dataset/bmw.csv
Parsing dataset/hyundi.csv
Parsing dataset/merc.csv
Parsing dataset/vauxhall.csv


## Corrections typographiques

In [16]:
# brand typo correction
all_df['hyundai'] = all_df.pop('hyundi')
all_df['mercedes'] = all_df.pop('merc')
all_df['opel'] = all_df.pop('vauxhall')

## Séparation en différents ensembles de données

In [17]:
for brand, dataframe in all_df.items():
    print(f"Source: {brand}, Shape: {dataframe.shape}")
    # Add brand name as feature
    all_df[brand]['brand'] = brand
    
    # merge column 'tax' and 'tax(£)'
    if brand == 'hyundai':
        all_df[brand]['tax'] = all_df[brand]['tax(£)']
        all_df[brand].drop(labels='tax(£)',axis=1,inplace=True)

    # sanitize `model` blank space
    all_df[brand]['model'] = all_df[brand]['model'].str.strip()
    # X = all_df[brand] because we keep the target for the EDA and we will drop it later 
    # using X = all_df[brand].drop(target)
    X_train, X_val, X_test, _, _, _= beda.train_val_test_split(X=all_df[brand],
                                                                               y=all_df[brand][target], 
                                                                               train_size=.75, 
                                                                               val_size=.15, 
                                                                               test_size=.1, 
                                                                               random_state=1,
                                                                               show=True)
    all_set_df = pd.concat([all_set_df,all_df[brand]])
    all_df_train = pd.concat([all_df_train,X_train])
    all_df_val = pd.concat([all_df_val,X_val])
    all_df_test = pd.concat([all_df_test,X_test])

print(f"train_set:{all_df_train.shape}\nval_set:{all_df_val.shape}\ntrain_set:{all_df_test.shape}\n")

Source: skoda, Shape: (6267, 9)
	X_train: (4700, 10)
	X_val: (940, 10)
	X_test: (627, 10)
	y_train: (4700,)
	y_val: (940,)
	y_test: (627,)
Source: ford, Shape: (17965, 9)
	X_train: (13473, 10)
	X_val: (2695, 10)
	X_test: (1797, 10)
	y_train: (13473,)
	y_val: (2695,)
	y_test: (1797,)
Source: vw, Shape: (15157, 9)
	X_train: (11367, 10)
	X_val: (2274, 10)
	X_test: (1516, 10)
	y_train: (11367,)
	y_val: (2274,)
	y_test: (1516,)
Source: audi, Shape: (10668, 9)
	X_train: (8001, 10)
	X_val: (1600, 10)
	X_test: (1067, 10)
	y_train: (8001,)
	y_val: (1600,)
	y_test: (1067,)
Source: toyota, Shape: (6738, 9)
	X_train: (5053, 10)
	X_val: (1011, 10)
	X_test: (674, 10)
	y_train: (5053,)
	y_val: (1011,)
	y_test: (674,)
Source: bmw, Shape: (10781, 9)
	X_train: (8085, 10)
	X_val: (1617, 10)
	X_test: (1079, 10)
	y_train: (8085,)
	y_val: (1617,)
	y_test: (1079,)
Source: hyundai, Shape: (4860, 9)
	X_train: (3645, 10)
	X_val: (729, 10)
	X_test: (486, 10)
	y_train: (3645,)
	y_val: (729,)
	y_test: (486,)
Sourc

In [18]:
all_set_df.to_csv(join(directory,'all_set.csv'))
all_set_df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuel_type', 'tax',
       'mpg', 'engine_size', 'brand'],
      dtype='object')

In [22]:

all_df_train.to_csv(join(directory,'train_set.csv'))
all_df_val.to_csv(join(directory,'val_set.csv'))
all_df_test.to_csv(join(directory,'test_set.csv'))